In [1]:
import numpy as np
import CHONK_XL as chxl
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import xsimlab as xs
import CHONK_cpp as ch
import zarr
import helplotlib as hpl
import helper
%matplotlib widget
%load_ext xsimlab.ipython

In [7]:


@xs.process
class CustomParameters:
    label_array = xs.variable(intent = 'out', dims = (('y','x'), ('node')))
    label_list = xs.any_object()
    CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
    nx = xs.foreign(chxl.ChonkBase, "nx")
    ny = xs.foreign(chxl.ChonkBase, "ny")
    dx = xs.foreign(chxl.ChonkBase, "dx")
    dy = xs.foreign(chxl.ChonkBase, "dy")
    
    active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
    landscape = xs.any_object()

    def initialize(self):
        # Instanciating the landscape
        self.landscape = helper.Landscape()

        # params for the landscapes dimensions

        # landscape.set_dimensions_from_res( nx = 100, ny = 100, dx = 200, dy = 200)
        self.landscape.set_dimensions_from_length(nx = self.nx, ny = self.ny, lx = self.nx * self.dx, ly = self.ny * self.dy)
#         self.landscape.set_boundaries_elevation(N = 1000, S = 0)
        self.landscape.set_boundaries_elevation(N = 0, S = 0)
        self.landscape.set_rel_distances(mountain_front = 0.65, normal_fault = 0.30)
        self.landscape.generate_uplift_4_StSt(U = 2e-4)
        self.landscape.generate_uplift_Normal_fault(Upos = 1e-4, Uneg = 2e-4, alpha_pos = 2e3, alpha_neg = 0.8e4)
#         self.landscape.add_pluton( dimless_X = 0.6, dimless_Y = 0.3, half_width = 5000,  half_heigth = 3000)
        self.label_list = []
    
        self.label_array = self.landscape.indices

        self.label_list.append(ch.label(0))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 3e-6;
        self.label_list[-1].Ks_modifyer = 1.2;
        self.label_list[-1].Kr_modifyer = 0.8;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.5;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.label_list.append(ch.label(1))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 3e-6;
        self.label_list[-1].Ks_modifyer = 1;
        self.label_list[-1].Kr_modifyer = 0.3;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.1;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.CHONK.initialise_label_list(self.label_list)
        self.CHONK.update_label_array(self.label_array.ravel())
        
        
@xs.process
class UpliftLandscape(chxl.Uplift):
	uplift_done = xs.variable(intent = "out")
	runner_done = xs.foreign(chxl.Runner, "runner_done")
	uplift = xs.variable(intent = 'out', dims = [('y','x'), ('node')])
	switch_time = xs.variable(intent = 'in')
	CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
	active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
	landscape = xs.foreign(CustomParameters, "landscape")

	def initialize(self):
		self.uplift = self.landscape.uplift_phase_1
# 		self.uplift[[-1,0],:] = 0
		self.uplift[0,:] = self.landscape.N_bound
		self.uplift[-1,:] = self.landscape.S_bound
		self.uplift[[-1,0],:] = 0
		self.done = False

	@xs.runtime(args=['step_delta','step_end'])
	def run_step(self, dt, timing):
		self.CHONK.add_external_to_surface_elevation_tp1(self.uplift.ravel() * dt)
		self.uplift_done = True
		
		if (timing > self.switch_time and self.done == False):
			self.uplift = self.landscape.uplift_phase_2
# 			self.uplift[[-1,0],:] = 0
			self.uplift[0,:] = self.landscape.N_bound
			self.uplift[-1,:] = self.landscape.S_bound
			self.uplift[[-1,0],:] = 0
			self.done = True
            
@xs.process
class UpliftLandscapeStSt(chxl.Uplift):
	uplift_done = xs.variable(intent = "out")
	runner_done = xs.foreign(chxl.Runner, "runner_done")
	uplift = xs.variable(intent = 'out', dims = [('y','x'), ('node')])
	switch_time = xs.variable(intent = 'in')
	CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
	active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
	landscape = xs.foreign(CustomParameters, "landscape")

	def initialize(self):
		self.uplift = self.landscape.uplift4StSt
		self.uplift[0,:] = self.landscape.N_bound
		self.uplift[-1,:] = self.landscape.S_bound
		self.uplift[[-1,0],:] = 0
		self.done = False

	@xs.runtime(args=['step_delta','step_end'])
	def run_step(self, dt, timing):
		self.CHONK.add_external_to_surface_elevation_tp1(self.uplift.ravel() * dt)
		self.uplift_done = True
        
@xs.process
class UpliftLandscapeNF1(chxl.Uplift):
	uplift_done = xs.variable(intent = "out")
	runner_done = xs.foreign(chxl.Runner, "runner_done")
	uplift = xs.variable(intent = 'out', dims = [('y','x'), ('node')])
	switch_time = xs.variable(intent = 'in')
	CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
	active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
	landscape = xs.foreign(CustomParameters, "landscape")
	dy = xs.foreign(chxl.ChonkBase, "dy") 

	def initialize(self):
		self.uplift = self.landscape.uplift_NF
		self.uplift[0,:] = self.landscape.N_bound
		self.uplift[-1,:] = self.landscape.S_bound
		self.uplift[[-1,0],:] = 0
		self.done = False

	@xs.runtime(args=['step_delta','step_end'])
	def run_step(self, dt, timing):
        
        
        
		self.CHONK.add_external_to_surface_elevation_tp1(self.uplift.ravel() * dt)
		self.uplift_done = True
		topo = self.CHONK.get_surface_elevation_tp1().reshape(self.uplift.shape)
		peuslo  = (topo[1,:] - topo[0,:])/self.dy
		topo[0,:][topo[0,:]< 0.0001] = topo[1,:][topo[0,:]< 0.0001] - 0.0001 * self.dy
		topo[0,:][topo[0,:]> 0.001] = topo[1,:][topo[0,:]> 0.001] - 0.001 * self.dy
		
		self.CHONK.set_surface_elevation_tp1(topo.ravel())
            
# landscape.uplift4StSt

In [8]:
model = xs.Model({"ChonkBase": chxl.ChonkBase,
                "Runner": chxl.Runner,
                "Topography": chxl.Topography,
#                 "Uplift": UpliftLandscapeStSt,
#                 "Uplift": UpliftLandscape,
                "Uplift": UpliftLandscapeNF1,
#                 "Uplift": chxl.Uplift,
                "Lake": chxl.Lake,
                "Precipitation": chxl.Precipitation,
#                 "DefaultParameters": chxl.DefaultParameters,
                "DefaultParameters": CustomParameters,
                "Flow": chxl.Flow,
                "Fluvial": chxl.Fluvial,
                "Hillslope": chxl.Hillslope
            })



In [9]:
ny,nx = 100,100
dy,dx = 200,200

In [10]:
# %create_setup model
import xsimlab as xs
time = np.arange(0,2e7,1000)
otime = time[::100]

init_z = np.load("./initial_topo_100_100.npy")
init_z = np.random.rand(ny,nx)
# init_z[0,:] = 1000
# init_z[0,:] = 0
U = np.zeros((ny,nx)) + 2e-4

Utime = 1e8
ds_in = xs.create_setup(
    model=model,
    clocks={
        'time': time,
        'otime': otime
    },
    master_clock='time',
    input_vars={
        'ChonkBase__dx': dx,
        'ChonkBase__dy': dy,
        'ChonkBase__nx': nx,
        'ChonkBase__ny': ny,
        'ChonkBase__depths_res_sed_proportions': 10,
        'ChonkBase__n_depth_sed_tracking': 50,
        'ChonkBase__boundary_conditions': "periodic_EW",
        'Topography__initial_elevation': init_z,
#         'Uplift__uplift': U,
        'Uplift__switch_time': Utime,
        'Lake__method': 'explicit',
        'Lake__evaporation': True,
        'Lake__evaporation_rate': 1,
        'Flow__threshold_single_flow': 1e24,
        'Precipitation__precipitation_rate': 0.7
    },
    output_vars={
        'Topography__topography': 'otime',
        'Topography__sed_height': 'otime',
        'Flow__Qw': 'otime',
        'Flow__water_balance_checker': 'otime',
        'Lake__lake_depth': 'otime',
#         'Fluvial__Qs': 'otime',
        'Hillslope__Qs': 'otime',
    }
)


In [11]:
zg = zarr.group("lake_from.zarr", overwrite=True)
with model,xs.monitoring.ProgressBar():
    out_ds = ds_in.xsimlab.run(store = zg)
#     out_ds = mod1.xsimlab.run()  
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0

             0% | initialize 

1.00584|25.4333|25.2857|0.901935|value:-17.375
1.00652|118.109|117.344|0.900932|value:-16.5556
1.01349|69.0957|68.1759|0.854148|value:-9.14228
1.01045|101.658|100.607|0.854148|value:-7.66538
1.00169|47.1818|47.1021|0.854148|value:-6.05542
1.00627|118.12|117.384|0.900066|value:-16.7839
1.01165|71.017|70.1994|0.85515|value:-9.11882
1.00938|103.472|102.511|0.85515|value:-7.74796
1.00057|47.4494|47.4223|0.85515|value:-6.15263
1.00603|118.187|117.478|0.899235|value:-17.0258
1.00973|132.966|131.684|0.815138|value:-12.9515
1.06959|73.285|68.517|0.825914|value:-27.5866
1.0101|71.7309|71.0139|0.856133|value:-9.09981
1.00848|103.78|102.907|0.856133|value:-7.83358
1.00579|118.508|117.826|0.898754|value:-17.3347
1.00915|134.464|133.244|0.812575|value:-12.9625
1.04584|55.6934|53.2522|0.874843|value:-21.1303
1.07201|72.4425|67.5765|0.824882|value:-27.9836
1.00846|73.6776|73.0594|0.857098|value:-9.08565
1.00756|104.667|103.882|0.857098|value:-7.92262
1.00556|118.645|117.989|0.898071|value:-17.6173
1.

In [13]:
from ipyfastscape import TopoViz3d


app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Topography__topography" )

app.show()

Output(layout=Layout(height='640px'))

In [8]:
#np.save("initial_topo_100_100.npy", out_ds.Topography__topography.values[-1])
# np.save("sed4testHS_CHONK.npy", out_ds.Topography__sed_height.sel({'otime':1.001e6}, method="nearest").values)
# np.save("sed4testtopo_CHONK.npy", out_ds.Topography__topography.sel({'otime':1.001e6}, method="nearest").values)

In [9]:
fig,ax = plt.subplots()
ax.plot(out_ds.otime.values, out_ds.Flow__water_balance_checker.values/(dx*dy*nx*ny*0.3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
topo = out_ds.Topography__topography.sel({'otime':1e6}, method= "nearest").values

In [11]:
(topo[4,50] - topo[3,50])/200

-0.020201834463823048

In [12]:
out_ds.Topography__topography.sel({'otime':1e6}, method= "nearest")

<xarray.DataArray 'Topography__topography' (y: 100, x: 100)>
dask.array<getitem, shape=(100, 100), dtype=float64, chunksize=(25, 50), chunktype=numpy.ndarray>
Coordinates:
    otime    float64 1e+06
  * x        (x) float64 0.0 200.0 400.0 600.0 ... 1.94e+04 1.96e+04 1.98e+04
  * y        (y) float64 0.0 200.0 400.0 600.0 ... 1.94e+04 1.96e+04 1.98e+04

In [10]:
from importlib import reload  
import plotter as pol
import warnings;warnings.simplefilter('ignore')
reload(pol)

pol.anim_lake_cross_section(
	out_ds, # The input ds 
	fname = "outputgif",
	timedim = "otime", # the time dimension
	batch_dim = None, # if there is a batch dim to pick
	cross_section_dir = 'x', # is the cross section in x or y direction
	xy_cross_section = 10000, # coordinate on the other axis
	color_bedrock = 'gray', # color of the bedrock
	color_sediments = 'orange', # color of the bedrock
	color_water = 'blue', # color of the water
	z_min = None, # minimum z on the cross_section, if left to none -> min of all
	z_max = None, # max z on the cross_section, if left to none -> min of all
	# Map parameters
	cmap_elev = 'gist_earth', # cmap of the cross-section
	alpha_hillshade = 0.5, # transparency of the hillshade
    czmin = 0,
    czmax = 1500,
    figsize = (12,4.5),
    custom_tickszz = np.arange(0,21000,5000, dtype = np.int),
)

  0%|          | 0/300 [00:00<?, ?it/s]

Movie created at outputmp4.mp4
GIF created at outputgif.gif
